# MGIMO intensive

## Introduction to Computer Vision. Image processing with OpenCV

### 1. Library installation

Documentation for use of OpenCV with Python API [see here](https://docs.opencv.org/).

In [ ]:
!pip3 install opencv-python

In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

### 2. Explore the data

Take the dataset from Kaggle - [Country-Wise Nightlight Images Dataset](https://www.kaggle.com/datasets/abhijeetdtu/country-nightlight-dataset).

In [ ]:
DATA_PATH = '/home/jovyan/__DATA/mgimo_intensive/nlt'

In [ ]:
!ls -la $DATA_PATH

In [ ]:
IMG_PATH = '/home/jovyan/__DATA/mgimo_intensive/nlt/dataset'

In [ ]:
data_dir = Path(IMG_PATH)

In [ ]:
# 1. Count files and images
all_files = list(data_dir.glob('*'))
image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'}
image_files = [f for f in all_files if f.suffix.lower() in image_extensions]

print(f"Total files: {len(all_files)}")
print(f"Image files: {len(image_files)}")

In [ ]:
# 2. Calculate total size
total_size_bytes = sum(f.stat().st_size for f in all_files)
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Total size: {total_size_mb:.2f} MB")

In [ ]:
# 3. Identify image types
image_types = {}
for img_file in image_files:
    ext = img_file.suffix.lower()
    image_types[ext] = image_types.get(ext, 0) + 1

print("\nImage types:")
for ext, count in image_types.items():
    print(f"  {ext}: {count} images")

In [ ]:
# 4. Open and display a few images
if image_files:
    # Display up to 5 images
    num_to_show = min(4, len(image_files))
    
    fig, axes = plt.subplots(1, num_to_show, figsize=(15, 5))
    if num_to_show == 1:
        axes = [axes]
    
    for i, img_file in enumerate(image_files[:num_to_show]):
        # Read image with OpenCV
        img_path = str(img_file)
        img = cv2.imread(img_path)
        
        if img is not None:
            # Convert BGR to RGB for matplotlib
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Display image
            axes[i].imshow(img_rgb)
            axes[i].set_title(f"{img_file.name}\n{img.shape}")
            axes[i].axis('off')
        else:
            axes[i].text(0.5, 0.5, f"Could not load\n{img_file.name}", 
                       ha='center', va='center')
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print image details
    print("\nFirst few images details:")
    for img_file in image_files[:num_to_show]:
        img = cv2.imread(str(img_file))
        if img is not None:
            print(f"  {img_file.name}: shape={img.shape}, size={img_file.stat().st_size / 1024:.1f} KB")

### 3. Image processing

![Colour vision](imgs/rgb.png)

![Image array](imgs/imgarray.png)

#### 3.1. Open an image

In [ ]:
image_files[:3]

In [ ]:
img_file = str(image_files[3])

In [ ]:
img = cv2.imread(img_file)
assert img is not None, 'file could not be read, check if file exists'

In [ ]:
img

In [ ]:
type(img)

In [ ]:
img.shape

In [ ]:
# let's see our image

plt.figure(figsize=(16, 8))
plt.imshow(img)
plt.show()

In [ ]:
# access to one pixel
px = img[100, 100]
print(px)

In [ ]:
# accessing only blue pixel
blue = img[100, 100, 0]
print(blue)

#### 3.2. Basic operations

In [ ]:
# draw a box around player

img_city = img.copy()
cv2.rectangle(
    img_city,
    (5100, 3500),
    (5400, 3750),
    (0, 255, 0),
    2
)
plt.figure(figsize=(16, 8))
plt.imshow(img_city)
plt.show()

In [ ]:
# put some text on the image

cv2.putText(
    img_city,
    'This is a night city',
    (5100, 3400),
    cv2.FONT_HERSHEY_SIMPLEX,
    4,
    (0, 255, 0),
    4
)
plt.figure(figsize=(16, 8))
plt.imshow(img_city)
plt.show()

In [ ]:
# get certain regions of images
# take our box
# (5100, 3500)
# (5400, 3750)

city = img_city[3500:3750, 5100:5400]
plt.imshow(city)
plt.show()

In [ ]:
# image resize

height, width = city.shape[:2]
result = cv2.resize(
    city,
    (2 * width, 2 * height),
    interpolation=cv2.INTER_CUBIC
)
plt.imshow(result)
plt.show()

In [ ]:
# affine transformation
# all parallel lines in the original image
# will still be parallel in the output image
# we need three points from the input image
# and their corresponding locations in the output image

rows, cols, ch = city.shape
pts1 = np.float32([
    [25, 25],
    [75, 25],
    [25, 50]
])
pts2 = np.float32([
    [10, 40],
    [50, 50],
    [15, 75]
])

M = cv2.getAffineTransform(pts1, pts2)
dst = cv2.warpAffine(city, M, (cols, rows))

plt.subplot(121)
plt.imshow(city)
plt.title('Input')
plt.subplot(122)
plt.imshow(dst)
plt.title('Output')
plt.show()

In [ ]:
# color histogram for the image

color = ('b', 'g', 'r')
for i, col in enumerate(color):
    histr = cv2.calcHist(
        [city],
        [i],
        None,
        [256],
        [0, 256]
    )
    plt.plot(histr, color=col)
    plt.xlim([0, 256])
plt.show()

#### 3.3. Advanced

In [ ]:
# Canny Edge detection

edges = cv2.Canny(city, 100, 200)
plt.imshow(edges)
plt.show()

In [ ]:
# Image Segmentation

gray = cv2.cvtColor(city, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(
    gray,
    0,
    255,
    cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU
)
plt.imshow(thresh)
plt.show()